# Coursera | Andrew Ng (05-week3)—序列模型和注意力机制


【第 5 部分-序列模型-第三周】在吴恩达深度学习视频基础上，笔记总结，添加个人理解。- ZJ
    
>[Coursera 课程](https://www.coursera.org/specializations/deep-learning) |[deeplearning.ai](https://www.deeplearning.ai/) |[网易云课堂](https://mooc.study.163.com/smartSpec/detail/1001319001.htm)


[CSDN]()：
   


---

# 序列模型和注意力机制（Sequence to sequence models）

## <font color=#0099ff>3.1 基础模型 (Basic models)

您好，欢迎参加本课程的最后一周也是深度学习微专业课程的最后一周。

在本周中，您将学到 seq2seq (sequence to sequece models)。 从**机器翻译**到**语音识别**,都取得了很大的作用。

先从最基础模型开始，之后会学到 集束搜索 （Beam search）和注意力模型（Attention Model），一直到最后的音频模型，比如语音。

![这里写图片描述](//img-blog.csdn.net/20180314093606731?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)    

**例子：**

假设你想输入一个法语句子，例如“Jane visite I'Afrique en septembre”。并且，你想把它翻译成英文句子“ Jane is visiting Africa in September”。

用 $x^{\langle 1 \rangle}$到 $x^{\langle 5 \rangle}$，表示输入的句子的单词，用$y^{\langle 1 \rangle}$到 $y^{\langle 6 \rangle}$来表示输出序列中的单词。

那么，如何训练一个新的神经网络来输入序列 x，并输出到序列 y ?

**Sequence to Sequence model ：**

- 首先，建立一个网络，称之为 编码网络（encoder network），它是一个 RNN 的结构，RNN 的单元可以是 GRU 也可以是 LSTM.
- 每次只向这个网络输入一个法语单词，将输入序列接收完毕后，RNN 网络会输出一个向量，来代表这个输入序列。
- 之后，建立一个解码网络（decoder network）,以编码网络的输出作为输入，之后它可以被训练为每次输出一个翻译后的单词。直到输出序列结尾或者句子结尾标记。则解码网络工作结束。（把每一步生成的单元传递到下一个单元中来测试，就像之前用语言模型合成文本时一样）

 深度学习在近期最卓越的成功之一就是：这个模型确实有效。（在给出足够的法语和英语文本的情况下，训练这个模型，通过输入一个法语句子来输出对应的英语翻译 ）

这个网络用简单的编码网络来对输入的法语句子进行编码，然后用一个解码网络来生成对应的英语翻译。


![这里写图片描述](//img-blog.csdn.net/20180314093618606?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

**图像描述（Image captioning ）:**

另一个与 seq2seq 类似的结构，被用来做图像描述（Image captioning ）。

- 给出一张猫的图片，它能自动输出该图片的描述：一只猫坐在椅子上。

如何训练这样的网络？通过输入图像来输出描述？

**方法步骤：**

- 在之前图像描述的课程中，我们知道如何将图片输入到卷积神经网络中，比如一个预训练的 AlexNet 结构，然后让其学习图片的编码（或者学习图片的一系列特征）
- 上述图片给出的是  AlexNet 结构 ，去掉最后的 Softmax 单元，预训练的 AlexNet 结构 会给你一个 4096 维的特征向量，向量表示的就是这张猫的图片。（这个预训练网络可以是图像的编码网络，得到一个 4096  维的向量，来表示这张图片）
- 接着把这个向量输入到 RNN 中，RNN 要做的就是生成图像的描述，每次生成一个单词，（一个个的生成单词序列）

在图像描述领域这个方法相当有效。

总结：

以上学习了 seq2seq model 是怎么运作的，以及 image to sequendce or image captioning 图像描述，这两个模型的运作方式是类似的。
但是，运作方式也有一些不同，主要体现在如何用语言模型合成新的文本，并生成对应序列的方面，一个主要的区别就是：我们期望的不是一个随机选取的翻译，而是最准确的翻译。

    
    
---
## <font color=#0099ff> 3.2 选择最有可能的句子(Picking the most likely sentence)

在 seq2seq machine translation model 机器翻译模型中，以及在第一周所用的语言模型之间，有相似的地方，但是也有许多重要的区别，Let's take a look .

![这里写图片描述](//img-blog.csdn.net/20180314100106276?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

可以把 机器翻译想象成是建立一个条件语言模型。(Machine translation as building a conditional language model)

- 如上图所示，语言模型（language model ）是我们在第一周所建立的模型，这个模型可以估计句子的可能性，（这就是语言模型所做的事情）也可以用于生成新的句子。
- 机器翻译模型（Machine translation）:如上图所示，分为编码网络和解码网络
- 可以发现，机器翻译模型的解码网络（紫色部分）与上面的语言模型非常几乎是一样的。
- 不同在于语言模型总是以 零向量开始，而机器翻译模型中的 编码网络会计算出一系列向量来表示句子的输入。所以称之为条件语言模型（conditional language model）相比于语言模型输出任意句子的概率 ,翻译模型会输出句子的英文翻译（取决于输入的法语句子）。

![这里写图片描述](//img-blog.csdn.net/20180314100116483?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70) 

**Finding the most likely translation:**   

   现在，想要通过模型将法语翻译成英文。

- 通过输入的法语句子，模型将会告诉你各种英文翻译的所对应的可能性（概率） 。
- 显然，我们不希望它随机的输出，如果你从这个分布中，进行取样，得到 p(y | x),可能取样一次，就能得到很好的翻译，但也可能得到截然不同的翻译。如上图所示，得到几种不同的翻译结果。
- 所以，使用这个模型来进行机器翻译时，并不是从得到的分布中，进行随机取样，而是要找到一个英语句子 y 使得条件概率最大化 
- 在开始机器翻译系统时，要做的一件事，就是想出一个算法，用来找出合适的 y 值，使得该项最大化，而解决这种问题通用的算法，就是 Beam search(束搜索)

![这里写图片描述](//img-blog.csdn.net/20180314100126880?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

**Why not a greedy search?为什么不用贪心搜索呢？**

**贪心搜索:**是一种来自计算科学的算法，生成第一个词的分布以后，它将会根据你的条件语言模型，挑选出一个最有可能的词，进入你的机器翻译模型中，在挑选出第一个词之后，它将会在挑选出最有可能的第二个词，....第三个词....这种算法叫做贪心算法。

**但是**，你真正需要的是一次性调选择出整个单词序列，来使得整体的概率最大化。

如上图所示，第一个翻译比第二个翻译更加简洁，但是贪心算法，在选出前两个单词（Jane is ）后，会选择更加常见的 going 而不是 visiting,从而得到的不是最佳的翻译。

- 假设你有 10000 个词，然后又一个 10 个单词的句子，那么就有 $10000^{10}$ 种可能性，所以计算每一种概率是不实际的，因为用一个近似的搜索算法，它会尽力的挑选句子 y 使得条件概率最大化。虽然不能保证，但已经足够了。

$$arg \space  max_{y} P(y^{\langle 1 \rangle},y^{\langle 2 \rangle},\dots,y^{\langle Ty \rangle})$$

总结：本节学到了，机器翻译时如何用来解决条件语言模型问题的，相比之间的语言模型，随机得到一个句子，这个模型可以得到一个最优可能的英语翻译。我们需要一个合适的搜索算法，下节讲解。

   
    
---
## <font color=#0099ff> 3.3 集束搜索 (Beam Search)  

在机器翻译中，给定输入如法语句子，  我们想要的是最有可能的翻译结果，而不是随机的，对于语音识别也是一样，给定一个输入的语音片段，我们期望得到最接近原意的文本翻译结果。

**Beam Search （集束搜索）**就是来解决这个问题。

![这里写图片描述](//img-blog.csdn.net/20180314100229824?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70) 

**  Beam Search Algorithm:**

- Beam Search 首先要做的就是挑选要输出的英语翻译中的第一个单词，如上图所示，列出 10000 个词的词汇表，
- 第一步，我们使用编码解码这样的网络来评估第一个单词的概率值，给定输入序列 x 即法语作为输入，第一个输出 y 的概率值是多少，集束搜索算法会考虑多个选择，会有一个参数 B，叫做 集束宽（beam width）这个例子中，将 B = 3,代表一次会考虑 3 个，（in  jane  september）若是 10 ,则一次考虑 10 个，


![这里写图片描述](//img-blog.csdn.net/20180314100145139?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)


-  第二步，已经选出了 （in  jane  september） 作为第一个单词，三个最可能的选择，beam search 接下来会针对每个第一个单词，考虑第二个单词是什么，如（in------a/aron/september.....zulu）为了评估第二个单词，我们使用上图中 绿色部分的编码网络 当解码网络输出为 $y^{\langle 1 \rangle}$为 in ,然后 in 再作为下一次的输入 再输出  $y^{\langle 2 \rangle}$ ，这样的网络就可以评估第二个单词的概率了 $P(y^{\langle 2 \rangle} | x,"in")$

	第一个和第二个单词对的概率：
	$$ P(y^{\langle 1 \rangle},y^{\langle 2 \rangle} | x)=P(y^{\langle 1 \rangle} | x)P(y^{\langle 2 \rangle} | x,y^{\langle 1 \rangle} )$$

- (jane--------)(september--------) 同理，如上图所示，最终得出 30000 个可能的概率，然后从中选出 概率最大的三个，


![这里写图片描述](//img-blog.csdn.net/20180314100154618?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

**Beam searcj (B=3):**

我们快速解释下，集束搜索的下一步，前面说过前几个单词，最可能的选择是 in september  和 jane is  和 jane visits ，对于每一对单词，先保存下来，作为给定输入 x(法语句子)情况下， y1 和 y2 的概率值。

像之前一样，我们考虑第三个单词是什么，如上图所示，循环下去，直到句子结束。

总结：已经学习了集束搜索是如何工作的，同时当然也有一些技巧和优化，使得集束算法更加高效。下节继续。

    
---
## <font color=#0099ff> 3.4 改进定向搜索 (Refinements to Beam Search) 

上节学习了基本的束搜索算法，这节会学到一些技巧，能使算法运行的更好。

![这里写图片描述](//img-blog.csdn.net/20180314100248131?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

**Length normalization(长度归一化):**

- 长度归一化就是对集束搜索算法，稍微调整的一种方式，来得到更好的结果。
- 如上图所示，第一个公式是乘积概率，计算这个公式时，每一个概率都是远小于 1  的，很多个小于 1 的数值乘起来，会得到极小的数字，从而造成数值下溢（数值下溢，就是数值太小了）。
- 因此，解决上面数值下溢的问题，如上图第二个公式所示，我们不会最大化这个乘积，而是取 log 值，转为最大化这个 log 求和的概率，在选择最有可能的句子 y 时，会得到同样的结果。所以通过取 log ,会得到一个数值上更稳定的算法。
- log 函数是严格的单调递增函数，最大化 $log P ( y | x )$与最大化 $P ( y | x )$ 是一样的，如果一个 y 值能使前者最大，就肯定能使后者 也取最大，所以常用的是取对数和。
- 对第二个公式进一步优化，可以使机器翻译效果更好，目前存在的问题是，（第二个公式可能更趋向于选择简短的翻译结果），因为短句子的概率，是由更少数量的小于 1 的数字乘积得到的。（若很多概率是负值，那么相加后则是更大的负值）
- 所以，第三个公式，我们不再最大化这个目标函数，而是进行归一化，通过除以翻译结果的单词数量，这样就是取每个单词的概数对数值的平均值了。这样明显减少了对输出长句子的惩罚。
- 在实践中，有个探索性的方法，相比于直接除以 $T_y$(输出句子的单词总数)，采用更柔和的方法 $T_y^{\alpha}$ 加上指数 $\alpha$（超参数）,如  $\alpha = 0.7$，因为 若$\alpha = 1$则是完全归一化，若 $\alpha = 0$ 则是完全没有归一化。（只是试探性的，并没有理论验证）


**总结：**如何运行束搜索算法？

- 当你运行，Beam search 时，你会看到很多长度 $T_y = 1,2,3,4......30 $的句子，当束宽 beam width = 3,你会记录所有这些可能的句子的长度，三个最有可能的选择。
- 然后，针对这些所有句子的可能输出句子，并用第三个公式进行评分，取概率最大的几个句子，然后对这些束搜索得到的句子，计算这个目标函数。
- 最后，从经过评估的这些句子中，挑选出在归一化的 log 概率目标函数上得分最高的一个。有时也叫做，归一化的对数似然目标函数。这就是最终输出的翻译结果。


![这里写图片描述](//img-blog.csdn.net/20180314100259844?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

**一些实现的细节：**

如何选择束宽 beam width B?

- B 越大 考虑的选择越多，找到的句子可能越好，但是 B 越大，算法的计算代价越大，因为要把很多可能选择保存起来。

lage B : better result, but slower

small B: wrose result,but faster

取决于不同的应用场景，如产品，科研，论文，取决于特定的应用，特定的领域。

B = 10 ，100 ，1000，3000

    
---
## <font color=#0099ff> 3.5 定向搜索的误差分析 (Error analysis in beam search) 

在之前的课程中，学习到误差分析，如何可以帮助你集中时间来做项目中最有用的工作，**束搜索算法**是一种近似搜索算法，也被称为**启发式搜索算法**（heuristic search algorithm）。

它不总是输出可能性最大的句子，而是记录着 B = 3, 10 ,100 种可能。

so ,如果束搜索算法出现错误会怎样呢？

本节将会学习到，误差分析和束搜索算法是如何相互起作用的，以及你怎么才能发现是 beam  search 出现问题，还是 RNN 模型出现问题。

**对束搜索算法进行误差分析：**

![这里写图片描述](//img-blog.csdn.net/20180314100315229?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

如上图所示，对法语进行翻译：

Human：人类的翻译，在开发集中，翻译的是很好的，标记为 $y^*$
Algorithm : 机器翻译的很糟糕，标记为 $ \hat{y}$

 模型主要分为两部分 ：
 
1.一个是神经网络模型（sequence to sequence model）成为 RNN model ,是个编码器 （encoder） 和解码器(decoder)
2.另一部分是束搜索算法，以某个 集束宽度 beam width B 运行 

如果你能找出造成这个不好的翻译的原因，确定是哪一部分，可以更好的解决问题。所以我们要找到到底是哪部分出现的问题。

- RNN 会计算 $P(y|x)$,事实证明，此时你能做的最有效的事情就是用这个模型，来计算 $P(y^*|x)$ 的同时也计算 $P(\hat{y}|x)$,然后比较一下，这两个值哪个能大，你就能够更清楚地将这个糟糕的翻译结果，归咎于 RNN 或者 beam search 算法。

![这里写图片描述](//img-blog.csdn.net/20180314100323355?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

我们继续探究一下其中的逻辑，我们将比较 $P(y^*|x)$  $P(\hat{y}|x)$ 两个值的大小。

Case 1:  $P(y^*|x)$  > $P(\hat{y}|x)$ 
  
   - Beam search 选择了 $\hat{y}$,但是 $y^*$包含 的 $P(y|x)$ 的值更大。
   - 结论：Beam search 实际上不能给你一个能使  $P(y|x)$  最大化的 y 值。

个人理解：首先模型得出的结果是  $P(y^*|x)$  > $P(\hat{y}|x)$ ，也就是 Beam search 应该选择  $y^*$， 但是 Beam search 却选择了 $\hat{y}$，那么这就是 Beam search  的错了。
	
Case 2:  $P(y^*|x)$ ≤  $P(\hat{y}|x)$ ，

 -    $y^*$ 的翻译比 $\hat{y}$好，但是 RNN 却预测出来  $P(y^*|x)$ ≤  $P(\hat{y}|x)$，
 - 结论：RNN model 错了。


一个细节，如果使用了长度归一化，那么要做的就不是比较 $P(y^*|x)$  $P(\hat{y}|x)$ 两种可能性的大小，而是比较长度归一化后的最优化目标函数值，不过目前先忽略这种情况。

![这里写图片描述](//img-blog.csdn.net/20180314100332987?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

**错误分析过程：**

- 先遍历开发集，然后在其中找出算法产生的错误，比较 可能性大小，确定是哪种错误。
- 接着遍历更多的例子，如上图所示，逐一进行标记。
- 通过这个过程，就可以执行误差分析，得出束搜索算法和 RNN 模型出错的比例是多少。
- 有了这样的误差分析过程，你就可以对开发集中每一个糟糕的翻译，确定是 beam search 出了问题，还是 RNN model 出了问题。
- 确定下来，那个错误发生的比例大，然后再进行下一步的分析。

总结：

这就是束搜索算法中的误差分析，这个误差分析过程是十分有用的，它可以用于分析近似最佳算法 （approximate optimization algorithm），（如 beam search）,这些算法被用来优化学习算法（例如序列到序列模型/ RNN）输出的目标函数或代价函数。

    
---
## <font color=#0099ff> 3.6 Bleu 得分 (Bleu Score) 
 
机器学习的一大难题是，一个法语句子可以有多种英文翻译，而且都同样好，所以当有多个，同样好的答案时，怎么评估一个机器翻译系统呢？有多个答案的时候，要怎样衡量准确性呢？

常见的解决办法就是，通过 Bleu 来解决。接下来讲解 Bleu 怎么工作的。

![这里写图片描述](//img-blog.csdn.net/20180314100348691?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

如图所示：

- 法语句子 French: Le chat est sur le tapis.
- 人工翻译 参考 1：The cat is on the mat.
- 另一个人工翻译 参考2 ：There is a cat on the mat.

Bleu 所做的就是给定一个机器生成的翻译，它能够自动的计算一个分数，来衡量机器翻译的好坏，只要这个机器生成的翻译与任何一个人工翻译的结果足够接近，就能得到一个比较高的 Bleu 得分。

**Bleu : bilingual evaluation understudy (双语评估替补)**

Bleu 的初衷是相对于请评估员，人工的评估机器翻译系统，Bleu 得分就相当于一个候补者，它可以代替人类来评估机器翻译的每一个输出结果。

MT (machine translation 缩写)

- MT output: the the the the the the the  . 一个非常糟糕的翻译。

- 衡量输出质量的方法之 一 是，观察输出结果的每一个词。看其是否出现在参考中，这被称作是机器翻译的精确度。

- 在这种情况下  the the the the the the the 这 7 个单词，每个单词都出现在了 上面两个 人工翻译的句子中，所以 是 衡量结果是 7/7 。（“看起来是一个极好的精确度” 实际不然）

- Precision 衡量：7/7 （得分看着好，其实翻译是不好的）
- 所以这就是为什么把出现在参考中的词，在 MT 输出的所有词中所占的比例作为精确度评估标准，并不是很有用的原因。

**解决办法：**

改良后的精确度评估方法。

- 我们把每一个单词的记分上限，定为它在参考句子中出现的最多次数，在参考 1 中，单词 the 出现了两次，在参考 2 中，单词 the 只出现了 1 次。所以 单词 the  的得分上限为 2 。所以句子得分是 2/7

**Modified precision:** 2/7

- 分母 7 代表是 单词 the 出现的总次数，
- 分子 2 代表 单词 the 出现的次数，我们再到达上限时截断计数，这就是改良后的精确度评估。

目前我们关注的是单个单词的 Bleu 得分，但实际上我们想关注的是 成对的单词或者 句子的得分。

![这里写图片描述](//img-blog.csdn.net/20180314100358606?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

Bleu score on bigrams (二元词组的 Bleu 得分)：

bigrams 相邻的两个单词。

接着上面的例子，MT output 输出的 The cat the cat on the mat. 虽然不是很好的句子，但是比上一个强。

如上图所示，

1.对相邻的二元词组罗列下来，
2 count 然后计算 每个二元词组出现了多少次。
3.在定义 $Count_{clip}$截取计数最大上限，给算法设置得分上限，上限值为二元词组出现在参考 1 或 2 中的最大次数。
4.最后，修改后的二元词组的精确度就是，4/6 各列相加。
 

![这里写图片描述](//img-blog.csdn.net/20180314100408515?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

最后给出公式，如上图所示，


![这里写图片描述](//img-blog.csdn.net/20180314100421423?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)






    
---
## <font color=#0099ff> 3.7 注意力模型直观理解 (Attention Model Intuition) 


![这里写图片描述](//img-blog.csdn.net/20180314100452287?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)



![这里写图片描述](//img-blog.csdn.net/20180314100501443?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)



    
    
---
## <font color=#0099ff> 3.8 注意力模型 (Attention Model)  

![这里写图片描述](//img-blog.csdn.net/20180314100514992?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)



![这里写图片描述](//img-blog.csdn.net/20180314100526851?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)



![这里写图片描述](//img-blog.csdn.net/20180314100536352?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)





---
## <font color=#0099ff> 3.9 语音辨识 (Speech recognition)

![这里写图片描述](//img-blog.csdn.net/20180314100559707?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

![这里写图片描述](//img-blog.csdn.net/20180314100609751?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

![这里写图片描述](//img-blog.csdn.net/20180314100620278?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70) 
   
---
## <font color=#0099ff> 3.10 触发字检测 (Trigger word detection)
    
![这里写图片描述](//img-blog.csdn.net/20180314100641246?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)    
 
![这里写图片描述](//img-blog.csdn.net/20180314100651473?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)
   
---
## <font color=#0099ff> 3.11 结论和致谢 (Summary and thank you)  
    
    
 ![这里写图片描述](//img-blog.csdn.net/20180314100701827?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70) 
    
![这里写图片描述](//img-blog.csdn.net/2018031410071254?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

![这里写图片描述](//img-blog.csdn.net/20180314100718787?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L0pVTkpVTl9aSEFP/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)
    
---
 
**PS: 欢迎扫码关注公众号：「SelfImprovementLab」！专注「深度学习」，「机器学习」，「人工智能」。以及 「早起」，「阅读」，「运动」，「英语 」「其他」不定期建群 打卡互助活动。**

<center><img src="http://upload-images.jianshu.io/upload_images/1157146-cab5ba89dfeeec4b.jpg?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240"></center>